In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install implicit==0.4.4

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
import implicit
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
from platform import python_version  # Импорт функции для получения версии Python

# Получаем и выводим версию Python
print(python_version())  # Вывод версии Python


3.10.12


In [5]:
import sys  # Импорт модуля sys для работы с системой

print("Версия Python пользователя:", sys.version)  # Вывод версии Python


Версия Python пользователя: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [6]:
implicit.__version__

'0.4.4'

In [7]:
# Считываем данные из CSV-файла с помощью метода
data = pd.read_csv('/content/drive/MyDrive/rec_sys/lesson_2/webinar/webinar_2/retail_train.csv')

# Выводим первые строки данных
data.head()


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [8]:
# Подсчет количества отсутствующих значений (NaN) в каждом столбце датафрейма data
data.isna().sum()

user_id              0
basket_id            0
day                  0
item_id              0
quantity             0
sales_value          0
store_id             0
retail_disc          0
trans_time           0
week_no              0
coupon_disc          0
coupon_match_disc    0
dtype: int64

In [9]:
# Получение типов данных для каждого столбца в data
data.dtypes

user_id                int64
basket_id              int64
day                    int64
item_id                int64
quantity               int64
sales_value          float64
store_id               int64
retail_disc          float64
trans_time             int64
week_no                int64
coupon_disc          float64
coupon_match_disc    float64
dtype: object

In [10]:
# Получение количества уникальных пользователей, уникальных товаров и общего числа взаимодействий
users = data.user_id.nunique()  # Количество уникальных пользователей
items = data.item_id.nunique()  # Количество уникальных товаров
interactions = data.shape[0]   # Общее количество взаимодействий (количество строк в датафрейме)

# Вывод полученных значений
print('#Количество пользователей (users): ', users)
print('#Количество товаров (items): ', items)
print('#Общее количество взаимодействий (interactions): ', interactions)


#Количество пользователей (users):  2499
#Количество товаров (items):  89051
#Общее количество взаимодействий (interactions):  2396804


In [11]:
# Определяем размер тестовой выборки в неделях
test_size_weeks = 3

# Создаем датафрейм для обучающей выборки, выбирая данные до последних `test_size_weeks` недель
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]

# Создаем датафрейм для тестовой выборки, выбирая данные за последние `test_size_weeks` недель
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]


In [12]:
# Группировка тестового набора данных по 'user_id' и формирование массива уникальных 'item_id' для каждого пользователя
result = data_test.groupby('user_id')['item_id'].unique().reset_index()

# Переименование столбцов в новом DataFrame result
result.columns=['user_id', 'actual']


In [13]:
result.dtypes

user_id     int64
actual     object
dtype: object

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [14]:
def weighted_random_recommendation(items_probabilities, n=5):
    """
    Выполняет рекомендации на основе взвешенной случайной выборки.

    Аргументы:
    items_probabilities : pandas.DataFrame
        DataFrame с колонками 'item_id' и 'probability', содержащий вероятности для каждого товара.

    n : int, optional (default=5)
        Количество рекомендаций.

    Возвращает:
    list
        Список item_id рекомендованных товаров.
    """

    # Получение списка item_id и соответствующих вероятностей из DataFrame items_probabilities
    items = np.array(items_probabilities['item_id'])
    probability = np.array(items_probabilities['probability'])

    # Выбор случайных товаров с учетом заданных вероятностей
    recs = np.random.choice(items, size=n, replace=False, p=probability)

    return recs.tolist()  # Возвращаем список рекомендованных товаров в виде списка


In [15]:
%%time

# Группировка данных по 'item_id' и вычисление суммы продаж ('sales_value') для каждого товара
items_probabilities = data_train.groupby('item_id')['sales_value'].sum().reset_index()

# Вычисление логарифма от суммы продаж для каждого товара и добавление в DataFrame items_probabilities
items_probabilities['log'] = np.log1p(items_probabilities['sales_value'])

# Вычисление общей суммы логарифмов сумм продаж
sum_ = items_probabilities['log'].sum()

# Вычисление вероятностей на основе логарифмов сумм продаж и их общей суммы
items_probabilities['probability'] = items_probabilities['log'] / sum_.sum()


CPU times: user 262 ms, sys: 40 ms, total: 302 ms
Wall time: 723 ms


In [16]:
# Вывод суммы столбца 'probability'
print(items_probabilities['probability'].sum())

1.0


In [17]:
# Добавляем новый столбец 'weighted_random_recommendation' в DataFrame `result`,
# используя функцию weighted_random_recommendation с весовыми вероятностями items_probabilities.
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_probabilities, 5))

# Выводим первые строки DataFrame `result` с новым столбцом.
result.head()


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1074004, 911025, 15596450, 8181553, 953860]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[998567, 899115, 5590009, 1004867, 1125382]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1117813, 1089045, 822066, 1098550, 7168651]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1550321, 8068702, 9932217, 882830, 9363905]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1129537, 9677943, 931911, 10456049, 6391244]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [18]:
def random_recommendation(items, n=5):
    # Преобразуем список items в массив numpy для удобства работы
    items = np.array(items)

    # Генерируем случайные рекомендации размером n из массива items без повторений
    recs = np.random.choice(items, size=n, replace=False)

    # Возвращаем рекомендации в виде списка
    return recs.tolist()


In [19]:
%%time

# Создаем массив items, содержащий уникальные идентификаторы элементов из столбца 'item_id' в data_train
items = data_train.item_id.unique()

# Применяем функцию random_recommendation к каждому значению столбца 'user_id' в DataFrame `result`,
# используя массив items для генерации 5 случайных рекомендаций (n=5).
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

# Выводим первые строки DataFrame `result` с добавленным столбцом 'random_recommendation'.
result.head()


CPU times: user 5.59 s, sys: 51.9 ms, total: 5.64 s
Wall time: 8.66 s


,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1074004, 911025, 15596450, 8181553, 953860]","[8204875, 7470099, 960040, 1111793, 13910420]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[998567, 899115, 5590009, 1004867, 1125382]","[5568972, 554622, 8019766, 883399, 891410]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1117813, 1089045, 822066, 1098550, 7168651]","[906387, 6796009, 1052443, 1616802, 901649]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1550321, 8068702, 9932217, 882830, 9363905]","[955643, 837821, 819308, 871985, 9420077]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1129537, 9677943, 931911, 10456049, 6391244]","[12605136, 2052318, 1745441, 895138, 9265791]"


In [20]:
def popularity_recommendation(data, n=5):
    # Группируем данные по 'item_id' и суммируем 'sales_value' для каждого элемента
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()

    # Сортируем элементы по суммарным продажам ('sales_value') в порядке убывания
    popular.sort_values('sales_value', ascending=False, inplace=True)

    # Выбираем топ n элементов на основе суммарных продаж и извлекаем их 'item_id'
    recs = popular.head(n).item_id

    # Возвращаем рекомендации в виде списка
    return recs.tolist()


In [21]:
# Получаем популярные рекомендации на основе функции popularity_recommendation с n=5 для данных data_train
popular_recs = popularity_recommendation(data_train, n=5)

# Для каждого значения из столбца 'user_id' в DataFrame `result` применяем популярные рекомендации
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)


In [22]:
def precision(recommended_list, bought_list):
    # Преобразуем списки в массивы numpy для удобства работы
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    # Проверяем, содержатся ли элементы из recommended_list в bought_list
    flags = np.isin(bought_list, recommended_list)

    # Вычисляем долю фактически купленных элементов из рекомендованных
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    # Вычисляем precision только для первых k элементов из recommended_list
    return precision(recommended_list[:k], bought_list)


In [23]:
# Проходим по столбцам DataFrame `result` начиная с первого (индекс 1)
for name_col in result.columns[1:]:
    # Вычисляем среднюю точность (precision) для каждого столбца, используя функцию precision_at_k
    avg_precision = round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(), 5)

    # Выводим среднюю точность и имя столбца
    print(f"{avg_precision}:{name_col}")


1.0:actual
0.00137:weighted_random_recommendation
0.00059:random_recommendation
0.15524:popular_recommendation


In [24]:
result.head()

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1074004, 911025, 15596450, 8181553, 953860]","[8204875, 7470099, 960040, 1111793, 13910420]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[998567, 899115, 5590009, 1004867, 1125382]","[5568972, 554622, 8019766, 883399, 891410]","[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1117813, 1089045, 822066, 1098550, 7168651]","[906387, 6796009, 1052443, 1616802, 901649]","[6534178, 6533889, 1029743, 6534166, 1082185]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1550321, 8068702, 9932217, 882830, 9363905]","[955643, 837821, 819308, 871985, 9420077]","[6534178, 6533889, 1029743, 6534166, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1129537, 9677943, 931911, 10456049, 6391244]","[12605136, 2052318, 1745441, 895138, 9265791]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [25]:
# Группируем данные по 'item_id' и суммируем количество проданных товаров ('quantity') для каждого элемента
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()

# Переименовываем столбец 'quantity' в 'n_sold' для более ясного понимания содержимого
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

# Выводим первые строки DataFrame `popularity` для просмотра результатов
popularity.head()


,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [26]:
# Сортируем DataFrame `popularity` по столбцу 'n_sold' в порядке убывания
popularity = popularity.sort_values('n_sold', ascending=False).reset_index(drop=True)

# Сбрасываем индексы строк с сохранением отсортированного порядка для вывода первых строк DataFrame
popularity.head()


,item_id,n_sold
0,6534178,190227964
1,6533889,15978434
2,6534166,12439291
3,6544236,2501949
4,1404121,1562004


In [27]:
# Сортируем DataFrame `popularity` по столбцу 'n_sold' в порядке убывания и выбираем верхние 5000 элементов
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)['item_id'].tolist()

# Выводим первые пять идентификаторов самых популярных товаров из списка top_5000
top_5000[:5]


[6534178, 6533889, 6534166, 6544236, 1404121]

In [28]:
# Заменяем идентификаторы товаров в столбце 'item_id' DataFrame `data_train` на значение 6666
# для тех строк, где идентификатор товара не содержится в списке top_5000
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666

# Выводим первые строки DataFrame `data_train` для просмотра изменений
data_train.head()


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [29]:
# Создаем матрицу пользователь-товар с помощью функции pivot_table из DataFrame `data_train`
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id',  # Размещаем пользователей в строках
                                  columns='item_id',  # Размещаем товары в столбцах
                                  values='quantity',  # Значения - количество покупок
                                  aggfunc='count',  # Агрегируем данные подсчетом количества покупок
                                  fill_value=0  # Заполняем пропущенные значения нулями
                                 )


In [30]:
# Выводим user_item_matrix
user_item_matrix

item_id,6666,202291,397896,420647,480014,545926,707683,731106,818980,819063,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,707,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,254,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,164,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,138,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,422,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2497,468,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2498,332,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# Заменяем все ненулевые значения в матрице user_item_matrix на 1, чтобы обозначить факт взаимодействия пользователя с товаром
user_item_matrix[user_item_matrix > 0] = 1

# Приводим тип данных всех элементов матрицы user_item_matrix к типу float
user_item_matrix = user_item_matrix.astype(float)

# Создаем разреженную матрицу csr_matrix (compressed sparse row format) из user_item_matrix
# Это используется для эффективного хранения и работы с большими объемами данных
sparse_user_item = csr_matrix(user_item_matrix)


In [32]:
# Выводим user_item_matrix
user_item_matrix

item_id,6666,202291,397896,420647,480014,545926,707683,731106,818980,819063,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Вычисляем общую сумму всех значений в матрице user_item_matrix
total_sum = user_item_matrix.sum().sum()

# Вычисляем общее количество элементов в матрице (количество строк * количество столбцов)
total_elements = user_item_matrix.shape[0] * user_item_matrix.shape[1]

# Вычисляем процент заполненности матрицы как отношение суммы значений к общему количеству элементов, затем умножаем на 100
fill_percentage = (total_sum / total_elements) * 100

# Выводим процент заполненности матрицы
fill_percentage


5.337131853341217

In [34]:
# Получаем массивы идентификаторов пользователей и товаров из индексов и столбцов user_item_matrix
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

# Создаем массивы числовых идентификаторов пользователей и товаров от 0 до длины соответствующих массивов
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

# Создаем словари, связывающие числовые идентификаторы с исходными идентификаторами
id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

# Создаем обратные словари, связывающие исходные идентификаторы с числовыми идентификаторами
itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))


In [35]:
%%time

# Создаем модель Item-Item Collaborative Filtering с параметрами: количество ближайших соседей (K) и количество потоков (num_threads)
model = ItemItemRecommender(K=5, num_threads=4)

# Обучаем модель на транспонированной матрице item-user (транспонирование необходимо для Item-Item CF)
model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

# Получаем рекомендации для конкретного пользователя (userid=userid_to_id[2]) с помощью обученной модели
recs = model.recommend(
    userid=userid_to_id[2],  # Идентификатор пользователя
    user_items=csr_matrix(user_item_matrix).tocsr(),  # Матрица user-item
    N=5,  # Количество рекомендаций
    filter_already_liked_items=False,  # Исключаем уже лайкнутые элементы
    filter_items=[itemid_to_id[6666]],  # Исключаем элементы с itemid 6666
    recalculate_user=False  # Не пересчитываем веса пользователя для следующей итерации
)


  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.64 s, sys: 20.6 ms, total: 2.66 s
Wall time: 1.98 s


In [36]:
# Для каждого пользователя из столбца 'user_id' в DataFrame `result`
# Вычисляем рекомендации товаров с помощью обученной модели для данного пользователя
# Записываем рекомендации в столбец 'itemitem' в виде списка идентификаторов рекомендованных товаров
result['itemitem'] = result['user_id'].apply(lambda user_id: [
    id_to_itemid[rec[0]] for rec in model.recommend(
        userid=userid_to_id[user_id],  # Идентификатор пользователя
        user_items=sparse_user_item,   # Матрица user-item
        N=5,                           # Количество рекомендаций
        filter_already_liked_items=False,  # Исключаем уже лайкнутые элементы
        filter_items=[itemid_to_id[6666]],  # Исключаем элементы с itemid 6666
        recalculate_user=False  # Не пересчитываем веса пользователя для следующей итерации
    )
])


In [37]:
# Выводим первые три строки DataFrame `result`
result.head(3)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1074004, 911025, 15596450, 8181553, 953860]","[8204875, 7470099, 960040, 1111793, 13910420]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[998567, 899115, 5590009, 1004867, 1125382]","[5568972, 554622, 8019766, 883399, 891410]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1117813, 1089045, 822066, 1098550, 7168651]","[906387, 6796009, 1052443, 1616802, 901649]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]"


In [38]:
%%time

# Создаем модель CosineRecommender с параметрами: количество ближайших соседей (K) и количество потоков (num_threads)
model = CosineRecommender(K=1, num_threads=4)

# Обучаем модель на транспонированной матрице item-user (транспонирование необходимо для CosineRecommender)
model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

# Получаем рекомендации для конкретного пользователя (userid=userid_to_id[1]) с помощью обученной модели
recs = model.recommend(
    userid=userid_to_id[1],  # Идентификатор пользователя
    user_items=csr_matrix(user_item_matrix).tocsr(),  # Матрица user-item
    N=5,  # Количество рекомендаций
    filter_already_liked_items=False,  # Исключаем уже лайкнутые элементы
    filter_items=None,  # Не применяем фильтр для исключения определенных элементов
    recalculate_user=False  # Не пересчитываем веса пользователя для следующей итерации
)


  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.89 s, sys: 27 ms, total: 2.92 s
Wall time: 3.18 s


In [39]:
%%time

# Для каждого пользователя из столбца 'user_id' в DataFrame `result`
# Вычисляем рекомендации товаров с помощью обученной модели для данного пользователя
# Записываем рекомендации в столбец 'cosine' в виде списка идентификаторов рекомендованных товаров
result['cosine'] = result['user_id'].apply(lambda x: [
    id_to_itemid[rec[0]] for rec in model.recommend(
        userid=userid_to_id[x],  # Идентификатор пользователя
        user_items=sparse_user_item,  # Матрица user-item
        N=5,  # Количество рекомендаций
        filter_already_liked_items=True,  # Исключаем уже лайкнутые элементы
        filter_items=None,  # Не применяем фильтр для исключения определенных элементов
        recalculate_user=True  # Пересчитываем веса пользователя для следующей итерации
    )
])


CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 74.7 ms


In [40]:
# Выводим первые три строки DataFrame `result`
result.head(3)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1074004, 911025, 15596450, 8181553, 953860]","[8204875, 7470099, 960040, 1111793, 13910420]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[10455984, 12172240, 12810393, 15926844, 12324..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[998567, 899115, 5590009, 1004867, 1125382]","[5568972, 554622, 8019766, 883399, 891410]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[12810393, 13416351, 13945244, 15596279, 13842..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1117813, 1089045, 822066, 1098550, 7168651]","[906387, 6796009, 1052443, 1616802, 901649]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[12731436, 12731544, 13511722, 15926712, 13003..."


In [41]:
%%time

# Создание модели Item-Item Collaborative Filtering с параметрами: количество ближайших соседей (K) и количество потоков (num_threads)
model = ItemItemRecommender(K=2, num_threads=4)

# Обучение модели на транспонированной матрице item-user (транспонирование необходимо для Item-Item CF)
model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)


  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.62 s, sys: 28.3 ms, total: 2.65 s
Wall time: 3 s


In [42]:
%%time

# Для каждого пользователя из столбца 'user_id' в DataFrame `result`
# Вычисляем рекомендации товаров на основе собственных покупок с помощью обученной модели для данного пользователя
# Записываем рекомендации в столбец 'own_purchases' в виде списка идентификаторов рекомендованных товаров
result['own_purchases'] = result['user_id'].apply(lambda x: [
    id_to_itemid[rec[0]] for rec in model.recommend(
        userid=userid_to_id[x],  # Идентификатор пользователя
        user_items=sparse_user_item,  # Матрица user-item
        N=5,  # Количество рекомендаций
        filter_already_liked_items=False,  # Не исключаем уже лайкнутые элементы
        filter_items=[itemid_to_id[6666]],  # Исключаем элементы с itemid 6666
        recalculate_user=True  # Пересчитываем веса пользователя для следующей итерации
    )
])


CPU times: user 74.8 ms, sys: 983 µs, total: 75.8 ms
Wall time: 77.5 ms


In [43]:
def get_recommendations_for_user(user_id, model, user_item_matrix, id_to_itemid, itemid_to_id, num_rec=5):
    user_id_internal = userid_to_id[user_id]
    recommendations = model.recommend(
        userid=user_id_internal,
        user_items=user_item_matrix,
        N=num_rec,
        filter_already_liked_items=False,
        filter_items=[itemid_to_id[6666]],
        recalculate_user=True
    )
    return [id_to_itemid[rec[0]] for rec in recommendations]

result['func_purchase'] = result['user_id'].apply(
    lambda x: get_recommendations_for_user(
        x, model, sparse_user_item, id_to_itemid, itemid_to_id, num_rec=5
    )
)

In [44]:
# Вычисление средней метрики precision@k для каждого метода рекомендаций
# Проход по каждому столбцу (кроме первого) в DataFrame `result` и вычисление precision@k для каждой строки
# Затем вычисляется среднее значение precision@k для каждого метода рекомендаций
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(), 4)} : {name_col}")


1.0 : actual
0.0014 : weighted_random_recommendation
0.0006 : random_recommendation
0.1552 : popular_recommendation
0.151 : itemitem
0.0584 : cosine
0.2193 : own_purchases
0.2193 : func_purchase
